In [1]:
#Importing libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
#loading the train data and test data
train=pd.read_csv("Dataset/Train.csv")
test=pd.read_csv("Dataset/Test.csv")

In [3]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
train=train.dropna(subset=['Service'])

In [5]:
#finding categoritcal features
numeric_columns=train._get_numeric_data().columns
categorical_columns=list(set(train.columns)-set(numeric_columns))
categorical_columns.remove('Result')
categorical_columns

['Flag', 'Service', 'Protocol_type']

In [6]:
#converting categorical values into numeric 
#can be done in this way as well
#from sklearn import preprocessing
#label_encoder=preprocessing.LabelEncoder()
#train['Flag']=label_encoder.fit_transform(train['Flag'])
train.Protocol_type=train.Protocol_type.map({"tcp":0,"udp":1,"icmp":2})
train.Service=train.Service.map({"ftp_data":0,"other":1,"private":2,"http":3,"remote_job":4,"name":5,"netbios_dgm":6,"eco_i":7,"mtp":8,"telnet":9,"finger":10,"domain_u":11,"supdup":12,"uucp_path":13,"Z39_50":14,"smtp":15,"csnet_ns":16,"uucp":17,"netbios_dgm":18,"urp_i":19,"auth":20,"domain":21,"ftp":22,"bgp":23,"ldap":24,"ecr_i":25,"gopher":26,"vmnet":27,"systat":28,"http_433":29,"efs":30,"whois":31,"imap4":32,"iso_tsap":33,"echo":34,"klogin":35,"link":36,"sunrpc":37,"login":38,"kshell":39,"sql_net":40,"time":41,"hostnames":42,"exec":43,"ntp_u":44,"discard":45,"nntp":46,"courier":47,"ctf":48,"ssh":49,"daytime":50,"shell":51,"netstat":52,"pop3":53,"nnsp":54,"IRC":55,"pop_2":56,"printer":57,"tim_i":58,"pm_dump":59,"red_i":60,"netbios_ssn":61,"rje":62,"X11":63,"urh_i":64,"http_8001":65,"aol":66,"http_2784":67,"tftp_u":68,"harvest":69})
train.Flag=train.Flag.map({"SF":0,"S0":1,"REJ":2,"RSTR":3,"SH":4,"RSTO":5,"S1":6,"RSTOS0":7,"S3":8,"S2":9,"OTH":10})

# Model Building

In [7]:
train.columns

Index(['Duration', 'Protocol_type', 'Service', 'Flag', 'Src_bytes',
       'Dst_bytes', 'Land', 'Wrong_fragment', 'Urgent', 'Hot',
       'Num_failed_logins', 'Logged_in', 'Num_compromised', 'Root_shell',
       'Su_attempted', 'Num_root', 'Num_file_creations', 'Num_shells',
       'Num_access_files', 'Num_outbound_cmds', 'Is_hot_login',
       'Is_guest_login', 'Count', 'Srv_count', 'Serror_rate',
       'Srv_serror_rate', 'Rerror_rate', 'Srv_error_rate', 'Same_srv_rate',
       'Diff_srv_rate', 'Srv_diff_host_ rate', 'Dst_host_count',
       'Dst_host_srv_count', 'Dst_host_same\n_srv_rate',
       'Dst_host_diff_ srv_rate', 'Dst_host_same _src_port_rate',
       'Dst_host_srv_ diff_host_rate', 'Dst_host_serro r_rate',
       'Dst_host_srv_s error_rate', 'Dst_host_rerro r_rate',
       'Dst_host_srv_r error_rate', 'Result'],
      dtype='object')

In [8]:
predictors = train[train.columns.difference(['Result'])]

In [9]:
predictors.columns

Index(['Count', 'Diff_srv_rate', 'Dst_bytes', 'Dst_host_count',
       'Dst_host_diff_ srv_rate', 'Dst_host_rerro r_rate',
       'Dst_host_same\n_srv_rate', 'Dst_host_same _src_port_rate',
       'Dst_host_serro r_rate', 'Dst_host_srv_ diff_host_rate',
       'Dst_host_srv_count', 'Dst_host_srv_r error_rate',
       'Dst_host_srv_s error_rate', 'Duration', 'Flag', 'Hot',
       'Is_guest_login', 'Is_hot_login', 'Land', 'Logged_in',
       'Num_access_files', 'Num_compromised', 'Num_failed_logins',
       'Num_file_creations', 'Num_outbound_cmds', 'Num_root', 'Num_shells',
       'Protocol_type', 'Rerror_rate', 'Root_shell', 'Same_srv_rate',
       'Serror_rate', 'Service', 'Src_bytes', 'Srv_count',
       'Srv_diff_host_ rate', 'Srv_error_rate', 'Srv_serror_rate',
       'Su_attempted', 'Urgent', 'Wrong_fragment'],
      dtype='object')

In [10]:
predictors.shape

(125973, 41)

In [11]:
predictors.head()

,Count,Diff_srv_rate,Dst_bytes,Dst_host_count,Dst_host_diff_ srv_rate,Dst_host_rerro r_rate,Dst_host_same\n_srv_rate,Dst_host_same _src_port_rate,Dst_host_serro r_rate,Dst_host_srv_ diff_host_rate,Dst_host_srv_count,Dst_host_srv_r error_rate,Dst_host_srv_s error_rate,Duration,Flag,Hot,Is_guest_login,Is_hot_login,Land,Logged_in,Num_access_files,Num_compromised,Num_failed_logins,Num_file_creations,Num_outbound_cmds,Num_root,Num_shells,Protocol_type,Rerror_rate,Root_shell,Same_srv_rate,Serror_rate,Service,Src_bytes,Srv_count,Srv_diff_host_ rate,Srv_error_rate,Srv_serror_rate,Su_attempted,Urgent,Wrong_fragment
0,2,0.00,0,150,0.03,0.05,0.17,0.17,0.00,0.00,25,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1.00,0.0,0.0,491,2,0.00,0.0,0.0,0,0,0
1,13,0.15,0,255,0.60,0.00,0.00,0.88,0.00,0.00,1,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0,0.08,0.0,1.0,146,1,0.00,0.0,0.0,0,0,0
2,123,0.07,0,255,0.05,0.00,0.10,0.00,1.00,0.00,26,0.00,1.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0.05,1.0,2.0,0,6,0.00,0.0,1.0,0,0,0
3,5,0.00,8153,30,0.00,0.00,1.00,0.03,0.03,0.04,255,0.01,0.01,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0,1.00,0.2,3.0,232,5,0.00,0.0,0.2,0,0,0
4,30,0.00,420,255,0.00,0.00,1.00,0.00,0.00,0.00,255,0.00,0.00,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0,1.00,0.0,3.0,199,32,0.09,0.0,0.0,0,0,0


In [12]:
train.head()

,Duration,Protocol_type,Service,Flag,Src_bytes,Dst_bytes,Land,Wrong_fragment,Urgent,Hot,Num_failed_logins,Logged_in,Num_compromised,Root_shell,Su_attempted,Num_root,Num_file_creations,Num_shells,Num_access_files,Num_outbound_cmds,Is_hot_login,Is_guest_login,Count,Srv_count,Serror_rate,Srv_serror_rate,Rerror_rate,Srv_error_rate,Same_srv_rate,Diff_srv_rate,Srv_diff_host_ rate,Dst_host_count,Dst_host_srv_count,Dst_host_same\n_srv_rate,Dst_host_diff_ srv_rate,Dst_host_same _src_port_rate,Dst_host_srv_ diff_host_rate,Dst_host_serro r_rate,Dst_host_srv_s error_rate,Dst_host_rerro r_rate,Dst_host_srv_r error_rate,Result
0,0,0,0.0,0,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,1,1.0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,0,3.0,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,0,3.0,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [13]:
#allocating the targeted columns
X=predictors.values
#assigning target variable
Y=train.Result

In [14]:
X,Y

(array([[2.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
        [1.30e+01, 1.50e-01, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
        [1.23e+02, 7.00e-02, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
        ...,
        [1.00e+00, 0.00e+00, 3.84e+02, ..., 0.00e+00, 0.00e+00, 0.00e+00],
        [1.44e+02, 5.00e-02, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
        [1.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00]]),
 0          normal
 1          normal
 2         neptune
 3          normal
 4          normal
            ...   
 125968    neptune
 125969     normal
 125970     normal
 125971    neptune
 125972     normal
 Name: Result, Length: 125973, dtype: object)

In [15]:
#using train_test_split function from sklearn to divide the train dataset
#importing library
from sklearn.model_selection import train_test_split

In [16]:
#split the data into train and cross validation 
#same train and test sets across different executions hence random_state=0.
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [17]:
#shaping the dimension of the data
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((88181, 41), (37792, 41), (88181,), (37792,))

In [18]:
#importing decision tree and accuracy_score from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics


In [19]:
X_train = np.nan_to_num(X_train.astype(np.float32))
X_test = np.nan_to_num(X_test.astype(np.float32))

In [20]:
#fit the model
model=DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [21]:
#X_test = test[predictors].values
predicted= model.predict(X_test)

In [22]:
accuracy_score(y_test,predicted)

0.9969305673158341

In [23]:
print(metrics.classification_report(y_test,predicted,))
confusion_matrix=metrics.confusion_matrix(y_test,predicted)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_cla

                 precision    recall  f1-score   support

           back       0.99      1.00      1.00       308
buffer_overflow       0.67      0.55      0.60        11
      ftp_write       0.50      0.50      0.50         2
   guess_passwd       1.00      0.90      0.95        21
           imap       0.60      0.75      0.67         4
        ipsweep       0.99      1.00      1.00      1071
           land       0.75      0.60      0.67         5
     loadmodule       0.00      0.00      0.00         0
       multihop       0.50      0.33      0.40         3
        neptune       1.00      1.00      1.00     12339
           nmap       0.99      0.99      0.99       467
         normal       1.00      1.00      1.00     20161
           perl       0.33      1.00      0.50         1
            pod       1.00      1.00      1.00        64
      portsweep       0.99      0.99      0.99       934
        rootkit       0.00      0.00      0.00         4
          satan       0.99    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.sum() - (FP + FN + TP)

In [25]:
FP_rate=FP/(FP+TN)

In [26]:
FP_rate

array([8.00341479e-05, 7.94049919e-05, 2.64620270e-05, 0.00000000e+00,
       5.29268551e-05, 1.63394243e-04, 2.64641279e-05, 2.64606266e-04,
       2.64627272e-05, 2.75016697e-04, 1.33958473e-04, 1.98513981e-03,
       5.29226535e-05, 0.00000000e+00, 1.62786912e-04, 1.58780565e-04,
       4.08641404e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.46629693e-04, 0.00000000e+00])

In [27]:
from sklearn.decomposition import PCA

In [40]:
pca=PCA(n_components=40)

In [41]:
predictors = np.nan_to_num(predictors.astype(np.float32))

In [42]:
xpca=pca.fit_transform(predictors)

In [43]:
xpca.shape

(125973, 40)

In [44]:
X_trainpca,X_testpca,y_trainpca,y_testpca=train_test_split(xpca,Y,test_size=0.2,random_state=30)

In [45]:
X_trainpca.shape,X_testpca.shape,y_trainpca.shape,y_testpca.shape

((100778, 40), (25195, 40), (100778,), (25195,))

In [46]:
modelpca=DecisionTreeClassifier()
modelpca.fit(X_trainpca,y_trainpca)

DecisionTreeClassifier()

In [47]:
predictedpca= modelpca.predict(X_testpca)

In [48]:
accuracy_score(y_testpca,predictedpca)

0.9958325064496923